In [1]:
import json
import time
import pandas as pd 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()
# Check if the producer is connected
connected = producer.bootstrap_connected()
print("Producer connected:", connected)

Producer connected: True


In [9]:
df_green = pd.read_csv('https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz', 
                       compression='gzip')


C:\Users\Jonah\AppData\Local\Temp\ipykernel_3872\3116872628.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_green = pd.read_csv('https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz',


In [ ]:
#df_green.head(5)

In [11]:
topic_name = 'green-trips'

t0 = time.time()

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    #print(row_dict)
    
    message = {
        'lpep_pickup_datetime': row_dict['lpep_pickup_datetime'],
        'lpep_dropoff_datetime': row_dict['lpep_dropoff_datetime'],
        'PULocationID': row_dict['PULocationID'],
        'DOLocationID': row_dict['DOLocationID'],
        'passenger_count': row_dict['passenger_count'],
        'trip_distance': row_dict['trip_distance'],
        'tip_amount': row_dict['tip_amount']
    }
    producer.send(topic_name, value=message)
    #print(f"Sent: {message}")
    
t1 = time.time()

print(f'took {(t0 - t1):.2f} seconds')
    

took -48.47 seconds


In [ ]:
import pyspark
pyspark.__file__

In [12]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()


In [13]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
green_stream.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [ ]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)
    
    if first_row:
        print(first_row[0]) 
        
query = green_stream.writeStream.foreachBatch(peek).start()

In [ ]:
query.stop()

In [14]:
from pyspark.sql import types
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [15]:
green_stream = green_stream \
  .select(from_json(col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [ ]:
query = green_stream \
    .writeStream \
    .format("console") \
    .start() \

query.awaitTermination(5000)

In [ ]:
from pyspark.sql.functions import current_timestamp, window

# Add a timestamp column to the DataFrame
green_stream_with_timestamp = green_stream.withColumn("timestamp", current_timestamp())

# Perform the rest of the operations (group by, count, order by)
popular_destinations = green_stream_with_timestamp \
    .groupBy(
        window(col("timestamp"), "5 minutes"),
        col("DOLocationID")
    ) \
    .count() \
    .orderBy("count", ascending=False)

# Write the results to the console
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

# Wait for the streaming query to finish
query.awaitTermination(5000)